##Importando modulos de aprendizado

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

##Tratando os dados
Remodelamos x_train e x_test porque nossa CNN aceita apenas um vetor quadridimensional. O valor 60000 representa o número de imagens nos dados de treinamento, 28 representa o tamanho da imagem e 1 representa o número de canais. O número de canais é definido como 1 se a imagem estiver em escala de cinza e se a imagem estiver no formato RGB, o número de canais estiver definido como 3.

In [4]:
batch_size = 128
num_classes = 10
epochs = 12

# dimensoes das imagens
img_rows, img_cols = 28, 28

# dividindo o dataset em treino e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Também convertemos nossos valores de destino em matrizes de classe binária.
    #Y = 2 # o valor 2 representa que a imagem possui o dígito 2 
    #Y = [0,0,1,0,0,0,0,0,0,0] # A segunda posição do vetor é feita 1 

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


#Criando a CNN

In [5]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25)) # Desativa aleatoriamente alguns neurônios da rede, o que força os dados a encontrar novos caminhos. 
                         # Isso reduz o super ajuste

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(num_classes, activation='softmax'))  # Camadas que são usadas para a previsão de classe (0–9).






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#Treinando o modelo Sequencial do Keras

In [0]:
# compilamos o modelo com uma função de perda de entropia cruzada categórica, otimizador Adadelta e uma métrica de precisão. 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])  

# ajustamos o conjunto de dados ao modelo, ou seja, treinamos o modelo por 12 épocas.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


# avaliamos a perda e a precisão do modelo nos dados de teste e os imprimimos.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#Resumo da rede

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [10]:
(model.predict(x_test[222:]))

array([[7.5813754e-05, 5.2623868e-06, 9.8850274e-01, ..., 4.1897845e-05,
        1.1158529e-02, 8.7776361e-06],
       [2.2351698e-10, 3.1596233e-11, 3.5759186e-08, ..., 9.9999905e-01,
        2.0102491e-11, 9.4228773e-07],
       [2.6035488e-08, 9.9999702e-01, 1.9847278e-06, ..., 1.6533144e-07,
        3.9175699e-08, 7.2510858e-10],
       ...,
       [6.3612073e-14, 1.6166879e-10, 8.5212881e-13, ..., 1.7250971e-09,
        2.1646522e-09, 1.9578566e-07],
       [6.1998076e-06, 7.4839992e-09, 4.3393804e-08, ..., 2.7762976e-08,
        7.3102987e-05, 1.3201939e-08],
       [1.1218156e-07, 8.5264116e-09, 2.5356441e-09, ..., 1.3058881e-12,
        1.7064241e-09, 1.5097995e-09]], dtype=float32)